In [1]:
import numpy as np

# Real sinusoid

A real sinusoid in discrete time domain can be expressed by: $x[n] = A cos(2 \pi f n T + \phi)$ where, $x$ is the array of real values of the sinusoid, $n$ is an integer value expressing the time index, $A$ is the amplitude value of the sinusoid, $f$ is the frequency value of the sinusoid in Hz, $T$ is the sampling period equal to $1/f_s$, $f_s$ is the sampling frequency in Hz, and $\phi$ is the initial phase of the sinusoid in radians.

In [2]:
def genSine(A, f, phi, fs, t):
    """
    Inputs:
        A (float) = amplitude of the sinusoid
        f (float) = frequency of the sinusoid in Hz
        phi (float) = initial phase of the sinusoid in radians
        fs (float) = sampling frequency of the sinusoid in Hz
        t (float) = duration of the sinusoid (is second)
    Output:
        x (numpy array) = The generated sinusoid
    """
    
    t_array = np.arange(0, t, 1/fs)
    x = A*np.cos(2*np.pi*f*t_array + phi)
    x = x.reshape(len(x), 1)
    return x

In [3]:
A=1.0; f=10.0; phi=1.0; fs=50.0; t=0.1
output = genSine(A, f, phi, fs, t)
expected = np.array([0.54030231, -0.63332387, -0.93171798, 0.05749049, 0.96724906]).reshape([5,1])
assert(np.allclose(output, expected))

# Complex sinusoid

A complex sinusoid in discrete time domain can be expressed by: $\bar{x}[n] = Ae^{j(\omega nT + \phi)} = A cos(\omega nT + \phi) + j A sin(\omega nT + \phi)$ where, $\bar{x}$ is the array of complex values of the sinusoid, $n$ is an integer value expressing the time index, $A$ is the amplitude value of the sinusoid, $\omega$ is the frequency of the sinusoid in radians per second (equal to $2 \pi f$), $T$ is the sampling period equal $1/f_s$, $f_s$ is the sampling frequency in Hz and $\phi$ is the initial phase of the sinusoid in radians.

In [4]:
def genComplexSine(k, N):
    """
    Note that this formula is not the same as the one above

    Inputs:
        k (integer) = frequency index of the complex sinusoid of the DFT
        N (integer) = length of complex sinusoid in samples
    Output:
        cSine (numpy array) = The generated complex sinusoid (length N)
    """
    n = np.arange(0, N)
    omega = 2*np.pi*k/N
    cSine = np.exp(-1j*omega*n)
    cSine = cSine.reshape(len(cSine), 1)
    return cSine

In [5]:
k=1; N=5
output = genComplexSine(1, 5)
expected = np.array([1.0 + 0.j, 0.30901699 - 0.95105652j,-0.80901699 - 0.58778525j, 
                     -0.80901699 + 0.58778525j, 0.30901699 + 0.95105652j]).reshape([5,1])
assert(np.allclose(output, expected))

# Discrete Fourier Transform (DFT)

The N point DFT of a sequence of real values $x$ (a sound) can be expressed by:

$$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2 \pi k n /N} $$

where $n$ is an integer value expressing the discrete time index, $k$ is an integer value expressing the discrete frequency index, and $N$ is the length of the DFT.

In [6]:
def DFT(x):
    """
    Input:
        x (numpy array) = input sequence of length N
    Output:
        X (numpy array) = The N point DFT of the input sequence x
    """
    
    N = x.shape[0]
    X = []
    
    for k in range(0,N):
        cSine = genComplexSine(k, N)
        dot = np.dot(cSine.T, x)
        X.append(dot)

    X = np.concatenate(X, axis=0)
    return X
    
x = np.array([1, 2, 3, 4]).reshape([4,1])
expected =  np.array([ 10.0 + 0.0j, -2.+2.0j, -2.0 - 9.79717439e-16j, -2.0 - 2.0j]).reshape([4,1])
output = DFT(x)
assert(np.allclose(output, expected))

# Inverse Discrete Fourier Transform

The IDFT of a spectrum $X$ of length $N$ can be expressed by:

$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] e^{j 2 \pi k n /N}$$

where $n = 0, ..., N − 1$. Here, $n$ is an integer value expressing the discrete time index, $k$ is an integer value expressing the discrete frequency index, and $N$ is the length of the spectrum $X$.

In [7]:
def IDFT(X):
    """
    Input:
        X (numpy array) = frequency spectrum (length N)
    Output:
        x (numpy array) = The IDFT of the frequency spectrum X (length N)
    """
    N = X.shape[0]
    x = []
    
    for k in range(0,N):
        cSine = genComplexSine(-k, N)
        dot = np.dot(cSine.T, X)/N
        x.append(dot)

    x = np.concatenate(x, axis=0)
    return x
    

X = np.array([1 ,1 ,1 ,1]).reshape([4,1])
expected = np.array([1.0 +0.0j, -4.5924255e-17 +5.5511151e-17j, 0.000000e+00 +6.12323400e-17j, 
                     8.22616137e-17 +8.32667268e-17j]).reshape([4,1])

output = IDFT(X)
assert(np.allclose(output, expected))

# Magnitude spectrum
    
The magnitude of a complex spectrum $X$ is obtained by taking its absolute value: $|X[k]|$.

In [8]:
def genMagSpec(x):
    """
    Input:
        x (numpy array) = input sequence of length N
    Output:
        magX (numpy array) = The length N magnitude spectrum of the input sequence x

    """
    X = DFT(x)
    magX = np.empty(X.shape)
    for i in range(0, X.shape[0]):
        a = np.real(X[i,0])
        b = np.imag(X[i,0])
        
        magX[i,0] = np.sqrt(np.power(a,2) + np.power(b,2))
    
    return magX

x = np.array([1, 2, 3, 4]).reshape([4,1])
expected = np.array([10.0, 2.82842712, 2.0, 2.82842712]).reshape([4,1])
output = genMagSpec(x)
assert(np.allclose(output, expected))